# Yolo real-time accident detection

In [ ]:
#run this cell only
from collections import defaultdict
import math
import cv2
import numpy as np
from ultralytics import YOLO
from cap_from_youtube import cap_from_youtube 
import uuid
import datetime
import torch

#enter your video link
url="https://www.youtube.com/watch?v=CTQErltXZqM"#example

source_error=False
try:
    if 'yout' in url:#if the url is from youtube then using cap_from_youtube package to import video
        cap = cap_from_youtube(url, 'best')  
    else:
        vurl=url+"/video"
        cap = cv2.VideoCapture(vurl)
except:
    print("Camera_source_error")    
    source_error=True  

width  = cap.get(3)
height = cap.get(4)
print('width, height:', width, height)
video_area=width*height
if int(video_area) == 0:
    print("Camera_source_error")     
    source_error=True            

if not source_error:      
    try:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        #moderate accuracy but require medium hardware for realtime at 720p i.e RTX 4050 
        model = YOLO('yolov8m.pt')
        #more accurate results but requires good hardware for realtime at 720p i.e RTX 4070Ti 
        #model = YOLO('yolov8x.pt')
        model.to(device=device)
    except:
        print("YOLO_model_loading_error")
    track_history = defaultdict(lambda: [])# Store the track history
    speed_history = defaultdict(lambda: [])# Store the relative history
    frame_skipper=0

    obj=[-1,-1]

    while cap.isOpened() :
        # Break the loop if 'q' is pressed
        if cv2.waitKey(30) & 0xFF == ord("q"):
            cap.release()
            cv2.destroyAllWindows() 
            break

        success, frame = cap.read()
        if success:
            #tracking only cars,trucks,buses,motorcycle and have a confidence more than 0.7
            results = model.track(frame,conf=0.75, classes=[1,2,3,5,7],persist=True)
            boxes = results[0].boxes.xywh
            try:
                class_ids=results[0].boxes.cls
                track_ids = results[0].boxes.id.int().tolist()
            except:
                track_ids=None

            annotated_frame = results[0].plot()

            if track_ids is not None:
                for box, track_id ,class_id in zip(boxes, track_ids,class_ids):
                    min_dist=1000
                    x, y, w, h = box        
                    area=float(w)* float(h)
                    track = track_history[track_id]
                    track.append((float(x), float(y)))  # x, y center point
                    if len(track) > 15:  # retain 7 tracks for 7 frames 
                        track.pop(0)

                    for box1, track_id1 ,class_id1 in zip(boxes, track_ids,class_ids):
                        if track_id != track_id1:#skipping its own track id
                            x1, y1, w1, h1 = box1
                            area1=float(w1)* float(h1)
                            point=(float(x), float(y))
                            point1=(float(x1), float(y1))
                            dist=(math.dist(point,point1))/((math.sqrt(area)+math.sqrt(area1))/2)
                             #relative distance between the points       
                            if dist<min_dist:
                                min_dist=dist
                                obj[0]=int((class_id))
                                obj[1]=int((class_id1))
                    # Draw the tracking lines
                    points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                    distance_draw=math.dist(track[0],track[-1])
                    if distance_draw==0:
                        relative_speed=0
                    else:    
                        relative_speed=distance_draw/area

                    speed=speed_history[track_id]   
                    speed.append(relative_speed)
                    if len(speed) > 5:  # retain 5 tracks for 5 frames 
                        speed.pop(0)

                    speed_difference=speed[-1]-speed[0]

                    if speed_difference==0:
                        acceleration=0
                    else:    
                        acceleration=speed_difference /len(speed)  

                    #print( results[0].names)
                    cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=5)
                    cv2.putText(annotated_frame,str(int(relative_speed*10000)),(int(x-(w/2)), int((h/2)+y)),cv2.FONT_HERSHEY_SIMPLEX ,1, (0, 225, 0),2,cv2.LINE_AA)
                    cv2.putText(annotated_frame,str(int(abs(acceleration)*20000)),(int(x+(w/2)), int((h/2)+y)),cv2.FONT_HERSHEY_SIMPLEX ,1, (0, 0, 225),2,cv2.LINE_AA)
                    cv2.putText(annotated_frame,str(int(min_dist*100)),(int(x), int((h/2)+y)),cv2.FONT_HERSHEY_SIMPLEX ,1, (225,0, 25),2,cv2.LINE_AA)
                    #changing the values may vary the detection of accident
                    if ((relative_speed*10000)>7) and ((min_dist*100)<120)and ((abs(acceleration)*20000)>3):#during accident the car should have some speed and it should have some accident                            
                        if frame_skipper>0:
                            frame_skipper=frame_skipper-1
                            continue
                        else: 
                            frame_skipper=25#skips accident detection for 25 frames after an accident is detected

                        #cv2.putText(annotated_frame,"Accident",(int(x-(w/4)), int(y)),cv2.FONT_HERSHEY_SIMPLEX ,1, (12,0,225),2,cv2.LINE_AA)
                        accident_id = str(uuid.uuid1())#Gets a new unique id at every acciddent
                        cropped_image = annotated_frame[int(y-(h*1.5)):int(y+(h*1.5)), int(x-(w*1.5)):int(x+(w*1.5))]
                        cv2.imwrite("accident_{}.jpg".format(accident_id), annotated_frame)#saves the file in your loacl machine
                        cv2.imwrite("accident_{}.jpg".format(accident_id), cropped_image)#rewrite the file after cropped,if cropping is successful


            # Display the annotated frame
            cv2.imshow("YOLOv8 Tracking", annotated_frame)




# install packages for this project

In [ ]:
#if you want to run on CPU just install this
!pip install opencv-python
!pip install ultralytics
!pip install cap-from-youtube
!pip install youtube_dl

In [ ]:
#If you want to run on gpu
#install cuda 12.1 from nvidia if not present

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
!pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 --index-url https://download.pytorch.org/whl/cu121

In [ ]:
import torch
import torchvision
import torchaudio
print(torch.__version__)
print(torchvision.__version__)
print(torchaudio.__version__)
#if it shows like this then ok
#2.1.1+cu121
#0.16.1+cu121
#2.1.1+cu121
